In [2]:
# Ignore the warnings
import warnings
# warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# System related and data input controls
import os

# Auto reload of library
%reload_ext autoreload
%autoreload 2

# Python path
import sys
base_folder = 'DataScience'
location_base = os.path.join(os.getcwd().split(base_folder)[0], base_folder)
location_module = [os.path.join(location_base, 'Module')] 
for each in location_module:
    if each not in sys.path:
        sys.path.append(each)
        
from import_KK import *
DeviceStrategy_GPU()
from preprocessing_KK import *
from preprocessing_text_KK import * ##
from visualization_KK import * ##
from algorithm_textmining_KK import *
from algorithm_machinelearning_KK import *
from algorithm_deeplearning_KK import *
from evaluation_KK import *

# 하이퍼파라미터
DELETE_KEYWORD = ['100세', '거주환경']
CATEGORY_BK = ['경제', '사회', '문화', '국제']
CATEGORY_BK_Sub = ['경제>경제일반', '경제>국제경제', '경제>취업_창업',
                   '사회>노동_복지', '사회>사건_사고', '사회>사회일반', '사회>여성', '사회>장애인', '사회>의료_건강',
                   '문화>미술_건축', '문화>요리_여행', '문화>출판',
                   '국제>중국', '국제>유럽_EU', '국제>일본', '국제>미국_북미', '국제>중동_아프리카',
                   '국제>아시아', '국제>중남미', '국제>국제일반', '국제>러시아']
CATEGORY_CR = ['세계', '경제', '생활/문화', '오피니언', '사회', 'IT/과학']
COLNAME_CATEGORY = '일자'
COLNAME_MINING = '제목'
# SAVE_LOCATION = r'C:\Users\user\Desktop\Data'    # inu
SAVE_LOCATION = r'C:\Users\KK\Desktop\Data'    # home

Python Version:  3.9.18 (main, Sep 11 2023, 14:09:26) [MSC v.1916 64 bit (AMD64)]

Tensorflow Version:  2.10.1
Num of Physical GPUs Available:  0
Cuda is ready?  True 


Torch Version:  2.1.0+cpu
No GPU available, using the CPU instead.


In [4]:
def preprocessing_sentence_to_BERTinput(X_series, Y_series, tokenizer, 
                                        seq_len=128, batch_size=32, sampler='random'):
    # BERT 입력 형식에 맞게 변환
    sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in X_series]
    
    # 전처리
    tokens, masks, segments, targets = [], [], [], []
    for i in tqdm(range(len(X_series))):
        ## 변환
        token = tokenizer.encode_plus(X_series[i], max_length=seq_len,
                                      pad_to_max_length=True, truncation=True,
                                      return_attention_mask=True,
                                      add_special_tokens=True)
        ## 정리
        tokens.append(token['input_ids'])
        masks.append(token['attention_mask'])
        segments.append(token['token_type_ids'])
        targets.append(Y_series[i])
    
    # array 변환
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    targets = np.array(targets)
    
    # tensor 변환
    tokens = torch.tensor(tokens)
    masks = torch.tensor(masks)
    segments = torch.tensor(segments)
    targets = torch.tensor(targets)
    
    # pytorch dataloader 연결
    data = TensorDataset(tokens, masks, targets)
    if sampler == 'random':
        dataloader = DataLoader(data, sampler=RandomSampler(data), batch_size=batch_size)
    elif sampler == 'sequential':
        dataloader = DataLoader(data, sampler=SequentialSampler(data), batch_size=batch_size)
        
    return dataloader

In [5]:
SEQ_LEN = 128
BATCH_SIZE = 32

# Data Loading

In [16]:
train = pd.read_csv(os.path.join(os.getcwd(), 'Data', 'MovieReview_NSMC', 'ratings_train.txt'), sep='\t')
test = pd.read_csv(os.path.join(os.getcwd(), 'Data', 'MovieReview_NSMC', 'ratings_test.txt'), sep='\t')
train, test = train[~train.document.isnull()].reset_index().iloc[:,1:] , test[~test.document.isnull()].reset_index().iloc[:,1:]
X_train, Y_train = train['document'], train['label']
X_test, Y_test = test['document'], test['label']

In [17]:

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)    # 'bert-base-multilingual-cased', 'klue/roberta-base'
train_dataloader = preprocessing_sentence_to_BERTinput(X_train, Y_train, tokenizer,
                                                       seq_len=SEQ_LEN, batch_size=BATCH_SIZE)
test_dataloader = preprocessing_sentence_to_BERTinput(X_test, Y_test, tokenizer,
                                                       seq_len=SEQ_LEN, batch_size=BATCH_SIZE)

100%|██████████████████████████████████████████████████████████████████████████| 49997/49997 [00:06<00:00, 8047.74it/s]
